# Significant Theta Findings

In [ ]:
import os
import numpy as np
import pylab as plt
from pandas import read_csv

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## File parameters.
space = 'source'
model_name = 'revised'
label = 'dacc-lh'
analysis = 'stim'
freq = 'theta'
contrast = 'Interference'
baseline = (-0.5, -0.1)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/%s' %space
img_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/%s' %space
info = read_csv(os.path.join(root_dir, 'afMSIT_%s_info.csv' %space))

## Load source data.
npz = np.load(os.path.join(root_dir, 'afMSIT_%s_%s_%s_%s.npz' %(space,analysis,label,freq)))
data = npz['data']
times = npz['times']

## Load cluster results.
f = os.path.join(root_dir, 'results', '%s_%s_frequency_results.csv' %(model_name, analysis))
clusters = read_csv(f)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Initialize plotting
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Intialize figure.
fig = plt.figure(figsize=(13,5), dpi=300)
ax = plt.subplot2grid((1,1),(0,0))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot DBS Difference.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

if contrast == 'DBS':
    colors = ['#0571b0','#ca0020']
    labels = ['DBS OFF','DBS ON']
elif contrast == 'Interference':
    colors = ['#7b3294','#008837']
    labels = ['Non-Conflict','Conflict']

title_dict = {'dacc-lh':'Left dACC', 'dlpfc_5-lh':'Left VLPFC', 'dlpfc_5-rh':'Right VLPFC', }

for m, color, legend in zip([0,1],colors,labels):

    ix, = np.where(info.DBS==m)
    mu = data[ix].mean(axis=0)
    if analysis == 'stim': mu -= mu[(times >= baseline[0])&(times <= baseline[1])].mean()
    se = data[ix].std(axis=0) / np.sqrt(len(ix))
    ax.plot(times, mu, linewidth=7, color=color, label=legend)
    ax.fill_between(times, mu-se, mu+se, color=color, alpha=0.15)

## Plot significant clusters.
for ix in np.where((clusters.Label==label)&(clusters.Freq==freq)&
                   (clusters.Contrast==contrast)&(clusters.FDR<0.05))[0]:
    
    y1, y2 = -1, 2.5
    tmin, tmax = clusters.loc[ix,'Tmin'], clusters.loc[ix,'Tmax']
    ax.fill_between(np.linspace(tmin,tmax,1e3), y1, y2, color='k', alpha=0.2) 
    ax.set_ylim(y1,y2)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Add Flourishes
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

ax.legend(loc=2, fontsize=24, markerscale=2, frameon=False, borderpad=0, handletextpad=0.2, handlelength=1.5)
ax.set_xlabel('Time (s)', fontsize=34)
ax.set_ylabel('Power (dB)', fontsize=32)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_title(title_dict[label], fontsize=44)

if analysis == 'stim':
    y1, y2 = ax.get_ylim()
    for x,s in zip([0, 0.4, 1.127],['IAPS','MSIT','Resp']): 
        ax.text(x+0.02,-0.75,s,fontsize=24)
        ax.vlines(x,y1,y2,linewidth=5, linestyle='--',alpha=0.4)
    ax.set_ylim(-0.8,2.2)
elif analysis == 'resp':
    y1, y2 = ax.get_ylim()
    ax.text(0.02, y1+0.05,'Resp', fontsize=24)
    ax.vlines(0.0,y1,y2,linewidth=5, linestyle='--',alpha=0.4)
    ax.set_ylim(y1,2.2)
    
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

if analysis == 'stim':
    ax.axhline(y = -0.8, linewidth=4, color='k')
    ax.axvline(x = -0.5, linewidth=4, color='k')
elif analysis == 'resp':
    ax.axhline(y = y1, linewidth=4, color='k')
    ax.axvline(x = -1.0, linewidth=4, color='k')
    

plt.tight_layout()
# plt.show()
plt.savefig('/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/acnp2016/%s.png' %label)
plt.close('all')
print 'Done.'

# EMOTE Labels

In [ ]:
import os
from surfer import Brain
%matplotlib qt4

fs_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/freesurfs'
subj_dir = os.environ["SUBJECTS_DIR"]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Make brain.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

subject = "fscopy"
surf = "smoothwm"
label = 'dlpfc_5-rh'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## Load files.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
## Make brain
hemi = label.split('-')[-1]
brain = Brain(subject, hemi, surf)

## Add label
f = os.path.join(fs_dir,subject,'label','april2016','%s.label' %label)
brain.remove_labels()
brain.add_label(f, color='#AFFF94', alpha=1)

if 'pfc' in label: view = 'lateral'
else: view = 'medial'

if hemi == 'lh' and view == 'lateral': brain.show_view({'azimuth': 160, 'elevation': 80}, roll=90, distance=450)
elif hemi == 'rh' and view == 'lateral': brain.show_view({'azimuth': 20, 'elevation': 80}, roll=-85, distance=450)
elif hemi == 'lh' and view == 'medial': brain.show_view({'azimuth': 10, 'elevation': 100}, roll=-85, distance=450)
elif hemi == 'rh' and view == 'medial': brain.show_view({'azimuth': 170, 'elevation': 100}, roll=90, distance=450)

In [ ]:
brain.save_image('/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/acnp2016/%s_%s.png' %(surf,label))